In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# ─── PATHS ──────────────────────────────────────────────────────────
input_folder  = r"Input Folder"
output_folder = r"Destination Folder where you want to save the results"
# ────────────────────────────────────────────────────────────────────

# 5 metrics we will consider (Accuracy, F1, Precision, Recall, AUC)
METRIC_NAMES = ["Accuracy", "F1", "Precision", "Recall", "AUC"]
KEEP_IDS = [0, 1, 2, 3, 4]  # Columns for these 5 metrics


# ╔═══════════════════════════════════════════════════════════════╗
# ║ 1. COLLECT DATA FROM ALL ROTATION FILES                       ║
# ╚═══════════════════════════════════════════════════════════════╝
def collect_tables(folder):
    wide_rows, mean_rows = [], []

    for fname in os.listdir(folder):
        if not fname.lower().endswith(".xlsx"):
            continue
        tag = os.path.splitext(fname)[0]  # rotation label
        path = os.path.join(folder, fname)

        # Read Excel sheets and fill NaNs with 0
        mean_df = pd.read_excel(path, "Aggregated_Results").fillna(0)
        std_df = pd.read_excel(path, "Aggregated_Std_Results").fillna(0)

        dim_red = mean_df.iloc[:, 13]
        clf = mean_df.iloc[:, -1]

        # Extract 5 CV metrics and External metrics
        cv_mean = mean_df.iloc[:, KEEP_IDS].to_numpy()
        cv_std = std_df.iloc[:, KEEP_IDS].to_numpy()
        ext_mean = mean_df.iloc[:, [i + 6 for i in KEEP_IDS]].to_numpy()
        ext_std = std_df.iloc[:, [i + 6 for i in KEEP_IDS]].to_numpy()

        # Rows for Sheet 1 (Ranked_Models)
        for i in range(len(mean_df)):
            rec = {"Dimensionality_Reduction": dim_red.iat[i],
                   "Classifier": clf.iat[i]}
            # Raw numbers for scoring
            for j in range(5):
                rec[f"CV_Mean_{j + 1}"] = cv_mean[i, j]
                rec[f"CV_Std_{j + 1}"] = cv_std[i, j]
            # 10 mean ± std columns (rounded to 2 decimals)
            for j, m in enumerate(METRIC_NAMES + METRIC_NAMES):
                phase = "(CV)" if j < 5 else "(Ext)"
                mean_val = (cv_mean if j < 5 else ext_mean)[i, j % 5]
                std_val = (cv_std if j < 5 else ext_std)[i, j % 5]
                rec[f"{tag}_{m} {phase}"] = f"{mean_val:.2f} ± {std_val:.2f}"  # Rounded to 2 decimals
            wide_rows.append(rec)

        # Rows for Sheet 2 (Mean_CV_and_External)
        tmp = pd.DataFrame({
            "Dimensionality_Reduction": dim_red,
            "Classifier": clf,
        })
        for j, m in enumerate(METRIC_NAMES):
            tmp[f"CV_{m}"] = cv_mean[:, j]
            tmp[f"Ext_{m}"] = ext_mean[:, j]
        mean_rows.append(tmp)

    return pd.DataFrame(wide_rows), pd.concat(mean_rows, ignore_index=True)


# ╔═══════════════════════════════════════════════════════════════╗
# ║ 2. SHEET 1 – RANKED MODELS                                    ║
# ╚═══════════════════════════════════════════════════════════════╝
def build_rank_sheet(wide_df):
    mean_cols = [f"CV_Mean_{k}" for k in range(1, 6)]
    std_cols = [f"CV_Std_{k}" for k in range(1, 6)]

    grp = wide_df.groupby(["Dimensionality_Reduction", "Classifier"])
    stats = grp[mean_cols + std_cols].mean().reset_index()

    # Compute normalized score (0-2)
    mm = MinMaxScaler().fit_transform(stats[mean_cols])
    ss = MinMaxScaler().fit_transform(stats[std_cols])
    stats["Score"] = (mm + (1 - ss)).mean(axis=1)
    stats["Rank"] = stats["Score"].rank(ascending=False, method="dense").astype(int)
    stats["Average_CV"] = stats[mean_cols].mean(axis=1)

    metric_cols = [c for c in wide_df.columns if "(" in c]
    first_repr = (
        wide_df[["Dimensionality_Reduction", "Classifier"] + metric_cols]
        .groupby(["Dimensionality_Reduction", "Classifier"])
        .first()
        .reset_index()
    )

    fixed = ["Dimensionality_Reduction", "Classifier", "Score", "Rank", "Average_CV"]
    ranked = pd.merge(stats, first_repr,
                      on=["Dimensionality_Reduction", "Classifier"])[fixed + metric_cols]
    return ranked.sort_values("Rank")  # Sort by Rank


# ╔═══════════════════════════════════════════════════════════════╗
# ║ 3. SHEET 2 – MEAN CV & EXTERNAL + SCORE/RANK                  ║
# ╚═══════════════════════════════════════════════════════════════╝
def build_mean_sheet(mean_df, ranked_df):
    key_cols = ["Dimensionality_Reduction", "Classifier"]
    metric_cols = [f"CV_{m}" for m in METRIC_NAMES] + [f"Ext_{m}" for m in METRIC_NAMES]

    # Compute mean of each metric across all rotations
    agg = (
        mean_df.groupby(key_cols, sort=False)[metric_cols]
        .agg(['mean', 'std'])
        .reset_index()
    )

    # Flatten multi-level columns
    agg.columns = [f"{col[0]}_{col[1]}" if col[1] else col[0] for col in agg.columns]

    # Create mean ± std columns (rounded to 2 decimals)
    for m in metric_cols:
        agg[f"{m}"] = agg[f"{m}_mean"].round(2).astype(str) + " ± " + agg[f"{m}_std"].round(2).astype(str)
        agg.drop(columns=[f"{m}_mean", f"{m}_std"], inplace=True)

    # Merge Score & Rank from ranked_df
    agg = agg.merge(ranked_df[key_cols + ["Score", "Rank"]], on=key_cols, how="left")

    ordered = key_cols + ["Score", "Rank"] + metric_cols
    agg = agg.set_index(key_cols).loc[
        ranked_df[key_cols].set_index(key_cols).index
    ].reset_index()
    return agg[ordered].sort_values("Rank")


# ╔═══════════════════════════════════════════════════════════════╗
# ║ 4. MAIN                                                       ║
# ╚═══════════════════════════════════════════════════════════════╝
def main():
    wide_df, numeric_df = collect_tables(input_folder)
    if wide_df.empty:
        raise RuntimeError("No data collected – check folder and sheet names.")

    # Generate ranked table based on the Score
    ranked_df = build_rank_sheet(wide_df)

    # Generate the numeric summary for the Mean_CV_and_External sheet
    mean_df_out = build_mean_sheet(numeric_df, ranked_df)

    # Write everything into the output Excel file
    os.makedirs(output_folder, exist_ok=True)
    out_xlsx = os.path.join(output_folder, "Ranked_Unique_Model_Scores.xlsx")
    with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
        ranked_df.to_excel(writer, sheet_name="Ranked_Models", index=False)
        mean_df_out.to_excel(writer, sheet_name="Mean_CV_and_External", index=False)

    print(f"\n✅ Excel saved with two sheets:\n   • Ranked_Models\n   • Mean_CV_and_External\n→ {out_xlsx}")


if __name__ == "__main__":
    main()
